In [1]:
# Install all required packages
!pip install pytorch-fid lpips torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [3]:
# Upload your Exported_FID/Real and Exported_FID/Fake folders
from google.colab import files
from zipfile import ZipFile

# Upload ZIP files if needed
print("Please upload Real.zip and Fake.zip (you can zip the folders before uploading).")
uploaded = files.upload()

# Unzip Real and Fake folders
for fname in uploaded.keys():
    if fname.endswith(".zip"):
        with ZipFile(fname, 'r') as zip_ref:
            zip_ref.extractall("Exported_FID")
print("Files extracted.")


Please upload Real.zip and Fake.zip (you can zip the folders before uploading).


Saving Exported_FID.zip to Exported_FID.zip
Files extracted.


In [4]:
# Run FID
!python -m pytorch_fid Exported_FID/Real Exported_FID/Fake


Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 143MB/s]
100% 26/26 [08:25<00:00, 19.44s/it]
100% 26/26 [08:26<00:00, 19.47s/it]
FID:  2.5882688892309034


In [6]:
import os
import torch
import lpips
from PIL import Image
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_fn = lpips.LPIPS(net='alex').to(device)

transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

lpips_scores = []

real_imgs = sorted([f for f in os.listdir("Exported_FID/Real") if f.endswith(".png")])
fake_imgs = sorted([f for f in os.listdir("Exported_FID/Fake") if f.endswith(".png")])

for r_img, f_img in zip(real_imgs, fake_imgs):
    r_path = os.path.join("Exported_FID/Real", r_img)
    f_path = os.path.join("Exported_FID/Fake", f_img)

    r = transform(Image.open(r_path).convert("RGB")).unsqueeze(0).to(device)
    f = transform(Image.open(f_path).convert("RGB")).unsqueeze(0).to(device)

    score = lpips_fn(r, f).item()
    lpips_scores.append(score)

avg_lpips = sum(lpips_scores) / len(lpips_scores)
print("Average LPIPS Score:", round(avg_lpips, 4))


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
Average LPIPS Score: 0.0006


In [7]:
# Inception Score and KID require image tensors — only run if you load them as dataset.
# Skipped in this notebook due to simplicity.
print("Run Inception Score and KID only in PyTorch pipeline with image tensors.")


Run Inception Score and KID only in PyTorch pipeline with image tensors.


In [11]:
!pip install torch-fidelity


In [18]:
import torch
from torchvision import transforms
from PIL import Image
import os
from torch_fidelity import calculate_metrics

# Set paths to your image folders
input_fake = "Exported_FID/Fake"

# Run Inception Score only
metrics = calculate_metrics(
    input1=input_fake,
    isc=True,
    cuda=torch.cuda.is_available(),
    verbose=True
)

print("\nInception Score Results:")
print(metrics["inception_score_mean"], "(mean)")
print(metrics["inception_score_std"], "(std)")


Creating feature extractor "inception-v3-compat" with features ['logits_unbiased']
Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 140MB/s]
Extracting features from input1
Looking for samples non-recursivelty in "Exported_FID/Fake" with extensions png,jpg,jpeg
Found 1280 samples
/usr/local/lib/python3.11/dist-packages/torch_fidelity/datasets.py:16: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes())).view(height, width, 3)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:6


Inception Score Results:
9.118471942387394 (mean)
0.7169504263344133 (std)


Processing samples
Inception Score: 9.118471942387394 ± 0.7169504263344133


In [13]:
!pip install torch-fidelity  # Ensure torch-fidelity is installed
# Restart the kernel after running this cell if needed

In [16]:
!pip show torch-fidelity


Name: torch-fidelity
Version: 0.3.0
Summary: High-fidelity performance metrics for generative models in PyTorch
Home-page: https://www.github.com/toshas/torch-fidelity
Author: Anton Obukhov
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, Pillow, scipy, torch, torchvision, tqdm
Required-by: 


In [17]:
!python -m torch_fidelity --input1 Exported_FID/Real --input2 Exported_FID/Fake --isc


/usr/bin/python3: No module named torch_fidelity.__main__; 'torch_fidelity' is a package and cannot be directly executed


In [19]:
import torch
from torch_fidelity import calculate_metrics

# Set paths to image folders
input_real = "Exported_FID/Real"
input_fake = "Exported_FID/Fake"

# Run KID computation only
metrics = calculate_metrics(
    input1=input_real,
    input2=input_fake,
    kid=True,
    cuda=torch.cuda.is_available(),
    verbose=True
)

print("\nKernel Inception Distance (KID):")
print("KID Mean:", metrics["kernel_inception_distance_mean"])
print("KID Std Dev:", metrics["kernel_inception_distance_std"])


Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting features from input1
Looking for samples non-recursivelty in "Exported_FID/Real" with extensions png,jpg,jpeg
Found 1280 samples
Processing samples
Extracting features from input2
Looking for samples non-recursivelty in "Exported_FID/Fake" with extensions png,jpg,jpeg
Found 1280 samples
Processing samples
                                                                                 


Kernel Inception Distance (KID):
KID Mean: -0.0005612259634634897
KID Std Dev: 4.0094245739275546e-05


Kernel Inception Distance: -0.0005612259634634897 ± 4.0094245739275546e-05
